In [ ]:
from astropy.io import fits
import numpy as np

In [ ]:
from sherpa.astro import ui

import matplotlib.pyplot as plt

In [ ]:
ui.set_xsabund("aspl")

In [ ]:
# Lead default dataset with backgrounds
# Wont' use that now, but maybe I need it later for testing.
# I've types all the file names in here, so I might as well keep it for now.

ui.load_pha("R1", "../data/default/P0502370201R1S001SRSPEC1001.FIT")
ui.load_rmf("R1", "../data/default/P0502370201R1S001RSPMAT1001.FIT")
ui.load_bkg("R1", "../data/default/P0502370201R1S001BGSPEC1001.FIT")
ui.load_pha("R2", "../data/default/P0502370201R2S004SRSPEC1001.FIT")
ui.load_rmf("R2", "../data/default/P0502370201R2S004RSPMAT1001.FIT")
ui.load_bkg("R2", "../data/default/P0502370201R2S004BGSPEC1001.FIT")

for psf_frac in [50, 75, 90, 95, 98]:
    ui.load_pha(
        f"{psf_frac}_R1", f"../data/psf_{psf_frac}/P0502370201R1S001SRSPEC1001.FIT"
    )
    ui.load_rmf(
        f"{psf_frac}_R1", f"../data/psf_{psf_frac}/P0502370201R1S001RSPMAT1001.FIT"
    )
    ui.load_pha(
        f"{psf_frac}_R2", f"../data/psf_{psf_frac}/P0502370201R2S004SRSPEC1001.FIT"
    )
    ui.load_rmf(
        f"{psf_frac}_R2", f"../data/psf_{psf_frac}/P0502370201R2S004RSPMAT1001.FIT"
    )

In [ ]:
ui.set_analysis("wave")
ui.set_stat('cash')

In [ ]:
ui.plot_data("R1", yerrorbars=False, linestyle="solid")
ui.plot_data("50_R1", overplot=True, yerrorbars=False, linestyle="solid")
ui.plot_data("98_R1", overplot=True, yerrorbars=False, linestyle="solid")
# ui.plot_data("R2", overplot=True)
ax = plt.gca()
ax.set_xlim(21.3, 22.4)
ax.set_ylim(None, 0.015)

In [ ]:
from sherpa.astro.xspec import XSgaussian, XSconstant
from sherpa.astro import hc

In [ ]:
def make_triplet():
    r = XSgaussian("r")
    i = XSgaussian("i")
    f = XSgaussian("f")

    r.lineE = hc / 21.6
    i.lineE = hc / 21.8
    f.lineE = hc / 22.1

    for m in [r, i, f]:
        m.norm = 1e-5
        m.LineE.frozen = True
        m.sigma = 0.0001
        m.sigma.frozen = True

    const = XSconstant("const")
    const.factor = 2e-6
    return const + r + i + f


In [ ]:
t50 = make_triplet()
ui.set_source("R1", t50)

In [ ]:
ui.ignore(0, 21.3)
ui.ignore(22.3, None)
ui.notice(21.3, 22.3)

In [ ]:
t98 = make_triplet()
ui.set_source("98_R1", t98)
t50 = make_triplet()
ui.set_source("50_R1", t50)

In [ ]:
t98.parts[0].parts[0].parts[0].factor = 1e-7
ui.fit("98_R1")
ui.conf("98_R1")

In [ ]:
ui.fit("50_R1")
ui.conf("50_R1")

In [ ]:
ui.plot_fit("50_R1")
ui.plot_fit("98_R1", overplot=True)

In [ ]:
from sherpa.astro.io import _reconstruct_rmf

In [ ]:
def rmf_to_2dmatrix(rmf):
    """

    Not very efficient (double loops in Python).
    This is meant for a single conversion for plotting, not for numerical
    evaluation.
    """
    rrmf = _reconstruct_rmf(rmf)
    # find max length of a row
    # All written for the case of n_grp being a vector.
    # Could be a number,
    # And for the other being 2D vectors (could be number of 1d)
    # Still, I think it's better to start from _reconstruct_rmf
    # to enure consistency. It's easy to be off by one
    # On the other hand, if I start from the Sherpa arrays like in plots_for_expl
    # then I don't hve ot deal with the 1D/2D vectors
    # np.atleast2d should be able to make that simply though - unless it's "object" type
    max_len = np.zeros_like(rrmf["N_GRP"])
    for i, ngrp in enumerate(rrmf["N_GRP"]):
        max_len[i] = rrmf["F_CHAN"][i][ngrp - 1] + rrmf["N_CHAN"][i][ngrp - 1]

    matrix = np.zeros((len(rrmf["N_GRP"]), max_len.max()))
    for i, ngrp in enumerate(rrmf["N_GRP"]):
        for j in range(ngrp):
            chans = slice(
                rrmf["F_CHAN"][i][j], rrmf["F_CHAN"][i][j] + rrmf["N_CHAN"][i][j]
            )
            cum_n_chan = np.sum(rrmf["N_CHAN"][i][:j])
            matrix[i, chans] = rrmf["MATRIX"][i][
                cum_n_chan : cum_n_chan + rrmf["N_CHAN"][i][j]
            ]
    return matrix

In [ ]:
from sherpa.astro.io import read_rmf, read_pha
psf_fracs = [50, 75, 90, 95, 98]

In [ ]:
rmfs = []
phas = []
for psf_frac in psf_fracs:
    rmfs.append(read_rmf(f"../data/psf_{psf_frac}/P0502370201R1S001RSPMAT1001.FIT"))
    phas.append(read_pha(f"../data/psf_{psf_frac}/P0502370201R1S001SRSPEC1001.FIT"))

In [ ]:
# Need to start with the outermost region

for i in range(len(psf_fracs) - 1, 0, -1):
    phas[i].counts -= phas[i - 1].counts
    mat = rmf_to_2dmatrix(rmfs[i])
    # mat_base is the mat for the next iteration
    # Could save 50% runtime by keeping a reference
    mat_base = rmf_to_2dmatrix(rmfs[i - 1])
    diff = mat - mat_base
    rmfs[i].matrix = diff[mat.nonzero()]

In [ ]:
for i, psf_frac in enumerate(psf_fracs):
    ui.set_data(f"p{psf_frac}", phas[i])
    ui.set_rmf(f"p{psf_frac}", rmfs[i])

In [ ]:
ui.set_analysis("wave")
ui.ignore(0, 21.3)
ui.ignore(22.4, None)
ui.notice(21.3, 22.4)

ui.plot_data("p50")
ui.plot_data("p98", overplot=True)

In [ ]:
from astropy.table import Table
import astropy.units as u
from astropy.io import fits

src_list = "../data/psf_66/P0502370201R1S001SRCLI_0000.FIT"
reg_spatial = Table.read(src_list, hdu=f"RGS1_SRC1_SPATIAL")
reg_spatial["XDSP_CORR"].unit = u.rad

In [ ]:
# Changes with wavelength are pretty small. 
# Could interpolate what's in the regions to the wavelength I want,
# but seems that that's not actually needed here.
# Or could read directly from XMM calibration data
def xdisp(wave, src_list):
    # load file
    reg_spatial = Table.read(src_list, hdu=f"RGS1_SRC1_SPATIAL")
    reg_spatial["XDSP_CORR"].unit = u.rad
    for row in reg_spatial:
        if (np.min(row["LAMBDA"]) < wave) and (np.max(row['LAMBDA']) > wave):
            break
    return row, np.max(row['XDSP_CORR'])

In [ ]:
xdisps = []
for psf_frac in [50, 75, 90, 95, 98]:
    src_list = f"../data/psf_{psf_frac}/P0502370201R1S001SRCLI_0000.FIT"
    xdisps.append(xdisp(21.6, src_list)[1])
xdisps = np.array(xdisps)

In [ ]:
    rline = XSgaussian("r")
    iline = XSgaussian("i")
    fline = XSgaussian("f")

    rline.lineE = hc / 21.6
    iline.lineE = hc / 21.8
    fline.lineE = hc / 22.1

    for m in [rline, iline, fline]:
        m.norm = 1e-5
        m.LineE.frozen = True
        m.sigma = 0.0001
        m.sigma.frozen = True

    const = XSconstant("const")
    const.factor = 2e-6

In [ ]:
for i in range(len(xdisps)-1, 0, -1):
    xdisps[i] -= xdisps[i - 1]

In [ ]:
normed_xdisp  = xdisps / xdisps[0]

In [ ]:
for i, psf_frac in enumerate(psf_fracs):
    ui.set_source(f"p{psf_frac}", normed_xdisp[i] * const + rline + iline + fline)

In [ ]:
iline.norm=1e-5
const.factor = 1e-7

In [ ]:
ui.fit('p50')

In [ ]:
ui.fit(*[f"p{psf_frac}" for psf_frac in psf_fracs])

In [ ]:
#ui.plot_fit('p50')
#ui.plot_fit('p75', overplot=True)
ui.plot_fit('p98', overplot=True)

In [ ]:
1.3**3